# Train your first Deep Reinforcement Learning Agent

PYTHON 3.9

El objetivo de este notebook es crear un sistema para entrenar los algoritmos que sea claro y escalable. Así que voy a tratar el menor número de celdas posibles

In [1]:
import sys #-- lo he necesitado para importar el paquete
sys.path.append('..')
import gym_hearthstone

## Argumentos
He añadido argumentos al environment para que sea más sencillo hacer el entrenamiento:

    action_type: Define el espacio de acción que se va a crear. Ahora mismo solo hay tres implementados.
    
        * "random"  -> Solo una acción possible (Tomar una acción aleatoria de las que son posibles).
        * "type_rd" -> 6 acciones posibles Realiza una acción aleatoria del tipo de acción devuelto por el DQN (en caso de tener alguna carta) y si no tengo ninguna carta del tipo de acción que ha devuelto el DQN entonces hace una acción aleatoria de entre las disponibles). NO TIENE SENTIDO PORQUE CONFUNDE AL DQN
        * "type"    -> 6 acciones posibles (Realiza una acción aleatoria del tipo de acción devuelto por el DQN (en caso de tener alguna carta) y si no tengo ninguna carta del tipo de acción que ha devuelto el DQN no hace nada. A continuación vuelve a invocar al DQN para obtener otro tipo de acción).
        
        
        
    reward_mode: Define el reward que devuelve el environment:
    
        * "simple"    -> Solo hay reward cuando se gana o se pierde.
        * "penalize"  -> Se penalizan las acciones que no son válidas.
        * "incentive" -> Se premian las acciones que son válidas.
        * "complex"   -> Se penalizan las acciones que no son válidas y se premian las válidas.


    steps: Por ahora usa el numero de steps para el nombre del fichero de salida. 
    
        

In [2]:
import gymnasium as gym
import gym_hearthstone
from stable_baselines3 import DQN
import time
import os

In [3]:

steps = 1000
run = -1
env = gym.make('Hearthstone-v1', action_type = "random", reward_mode = "simple")

[fireplace.__init__]: Initializing card database
[fireplace.__init__]: Merged 26660 cards
[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player1 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Armor Up!')>)> targeting [Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Armor Up!')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Summon(<Summon.CARD>=<Hero ('Garrosh Hellscream')>)> targeting [Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.actions]: Player1 summons [<Hero ('Garrosh Hellscream')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deat

IMPLEMENTED CARDS: 8048
UNIMPLEMENTED CARDS: 18612
Implemented but Uncollectible: 6145
CardClass.DEATHKNIGHT amount of cards is: 0
CardClass.DRUID amount of cards is: 130
CardClass.HUNTER amount of cards is: 132
CardClass.MAGE amount of cards is: 128
CardClass.PALADIN amount of cards is: 128
CardClass.PRIEST amount of cards is: 119
CardClass.ROGUE amount of cards is: 123
CardClass.SHAMAN amount of cards is: 126
CardClass.WARLOCK amount of cards is: 118
CardClass.WARRIOR amount of cards is: 120
CardClass.DEMONHUNTER amount of cards is: 41
CardClass.NEUTRAL amount of cards is: 738
HearthstoneEnv - Version 0.2.0
Step.BEGIN_MULLIGAN
Step.MAIN_ACTION


In [4]:
run += 1
## Generar Modelo
model = DQN("MultiInputPolicy", env, verbose=1)

# Resetear env y stats
env.reset_stats()
start = time.time()
#env.reset()

## Entrenar modelo
model.learn(total_timesteps=steps, log_interval=4)
end = time.time()




C:\Users\jesus\anaconda3\envs\env_gym-hearthstone\lib\site-packages\stable_baselines3\common\buffers.py:564: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 5.84GB > 0.89GB
  warnings.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\jesus\anaconda3\envs\env_gym-hearthstone\lib\site-packages\gymnasium\utils\passive_env_checker.py:197: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
C:\Users\jesus\anaconda3\envs\env_gym-hearthstone\lib\site-packages\gymnasium\utils\passive_env_checker.py:210: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(
[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player1 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Armor Up!')>)> targeting [Player(name='Player1', her

------------------------
>>> Episode 0
------------------------
Step.BEGIN_MULLIGAN
Step.MAIN_ACTION
>>> HANDP1:  116 Sparring Partner
>>> HANDP1:  2370 Evasive Drakonid
>>> HANDP1:  7895 Ornery Direhorn
>>> HANDP1:  3760 Drakkari Enchanter
>>> HANDP2:  2825 Poison Seeds
>>> HANDP2:  3704 Deathspeaker
>>> HANDP2:  4777 Ironwood Golem
>>> HANDP2:  2848 The Coin
>>> HANDP2:  4784 Sewer Crawler


[fireplace.actions]: Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.entity]: Player1 draws <Minion ('Tar Lord')>
[fireplace.card]: <Minion ('Tar Lord')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.card]: <Minion ('Evasive Drakonid')> moves from <Zone.HAND: 3> to <Zone.DECK: 2>
[fireplace.entity]: Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>) shuffles their deck
[fireplace.entity]: Player1 begins turn 2
[fireplace.entity]: Player1 is now at 2 mana crystals
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.entity]: Player1 draws <Minion ('Lord of the Arena')>
[fireplace.card]: <Minion ('Lord of the Arena')> moves from <Z

...

>>> HANDP1:  3703 Saronite Chain Gang
>>> HANDP1:  7760 Living Monument
>>> HANDP1:  2519 Naturalize
>>> HANDP1:  3667 Furnacefire Colossus
>>> HANDP1:  67 Darnassus Aspirant
>>> HANDP1:  4577 Tomb Spider
>>> HANDP2:  7891 Swamp King Dred
>>> HANDP2:  5610 Bilefin Tidehunter
>>> HANDP2:  176 Evil Heckler
>>> HANDP2:  3638 Play Dead
>>> HANDP2:  2432 Scarlet Crusader
>>> FIELDP2:  7961 Bluegill Warrior
------------------------------------------
>>> Current Step 7, Global 473, Average 53
------------------------------------------
>>> Moves Dictionary:  {'choice': [], 'play_card': [[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 2, 0, None], [<Move.play_card: 4>, 4, None, None], [<Move.play_card: 4>, 5, None, None]], 'heropower': [[<Move.hero_power: 1>, None, None]], 'minion_attack': [], 'hero_attack': [], 'end_turn': [[<Move.end_turn: 0>]]}

>>> Before possible_actions RL agent 6:[[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 2, 0, None], [<Move.play_card: 4>, 4

[fireplace.actions]: <Minion ('Darnassus Aspirant')> triggering <TargetedAction: SpendMana(<SpendMana.AMOUNT>=1)> targeting [Player(name='Player1', hero=<Hero ('Malfurion Stormrage')>)]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player1 ends turn 6
[fireplace.entity]: Ending One-Turn effect: <Enchantment ('Extra Attack')>
[fireplace.card]: <Enchantment ('Extra Attack')> moves from <Zone.PLAY: 1> to <Zone.REMOVEDFROMGAME: 5>
[fireplace.entity]: Player2 begins turn 7
[fireplace.entity]: Player2 is now at 3 mana crystals
[fireplace.actions]: Player(name='Player2', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player2', hero=<Hero ('Rexxar')>)]
[fireplace.entity]: Player2 draws <Minion ('Annoy-o-Tron')>
[fireplace.card]: <Minion ('Annoy-o-Tron')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing 

>>> HANDP1:  3703 Saronite Chain Gang
>>> HANDP1:  7760 Living Monument
>>> HANDP1:  2519 Naturalize
>>> HANDP1:  3667 Furnacefire Colossus
>>> HANDP1:  4577 Tomb Spider
>>> FIELDP1 NOT IMPLEMENTED:  67 Darnassus Aspirant
>>> HANDP2:  7891 Swamp King Dred
>>> HANDP2:  5610 Bilefin Tidehunter
>>> HANDP2:  176 Evil Heckler
>>> HANDP2:  3638 Play Dead
>>> HANDP2:  2432 Scarlet Crusader
>>> FIELDP2:  7961 Bluegill Warrior
------------------------------------------
>>> Current Step 9, Global 475, Average 53
------------------------------------------
>>> Moves Dictionary:  {'choice': [], 'play_card': [], 'heropower': [], 'minion_attack': [], 'hero_attack': [[<Move.hero_attack: 3>, None, 0], [<Move.hero_attack: 3>, None, 1]], 'end_turn': [[<Move.end_turn: 0>]]}

>>> Before possible_actions RL agent 3:[[<Move.hero_attack: 3>, None, 0], [<Move.hero_attack: 3>, None, 1], [<Move.end_turn: 0>]]
>>> Already Selected Actions RL agent 0:[]
>>> After possible_actions RL agent 3:[[<Move.hero_attack: 3>

[fireplace.actions]: Player(name='Player1', hero=<Hero ('Malfurion Stormrage')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player1', hero=<Hero ('Malfurion Stormrage')>)]
[fireplace.entity]: Player1 draws <Spell ('Healing Touch')>
[fireplace.card]: <Spell ('Healing Touch')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player1 plays <Minion ('Saronite Chain Gang')> (target=None, index=None)
[fireplace.entity]: Player1 pays 4 mana
[fireplace.card]: <Minion ('Saronite Chain Gang')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Minion ('Saronite Chain Gang')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=None)> targeting [<Minion ('Saronite Chain Gang')>]
[fireplace.actions]: Activating <Minion ('Saronite Chain Gang')> action targeting None
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Empty stack, refresh

>>> HANDP1:  3703 Saronite Chain Gang
>>> HANDP1:  7760 Living Monument
>>> HANDP1:  2519 Naturalize
>>> HANDP1:  3667 Furnacefire Colossus
>>> HANDP1:  4577 Tomb Spider
>>> HANDP1:  1867 Healing Touch
>>> FIELDP1 NOT IMPLEMENTED:  67 Darnassus Aspirant
>>> HANDP2:  7891 Swamp King Dred
>>> HANDP2:  176 Evil Heckler
>>> HANDP2:  3638 Play Dead
>>> HANDP2:  2432 Scarlet Crusader
>>> HANDP2:  3211 Annoy-o-Tron
>>> FIELDP2:  7961 Bluegill Warrior
>>> FIELDP2:  5610 Bilefin Tidehunter
>>> FIELDP2:  5611 Ooze
------------------------------------------
>>> Current Step 10, Global 476, Average 53
------------------------------------------
>>> Moves Dictionary:  {'choice': [], 'play_card': [[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 2, 0, None], [<Move.play_card: 4>, 2, 1, None], [<Move.play_card: 4>, 2, 2, None], [<Move.play_card: 4>, 2, 3, None], [<Move.play_card: 4>, 3, None, None], [<Move.play_card: 4>, 4, None, None], [<Move.play_card: 4>, 5, 0, None], [<Move.play_card: 4

In [5]:
## Calcular estadísticas
env.display_stats()

total_time = end-start

if(env.curr_episode > 0):
    avg_time = (end-start) / env.curr_episode
    avg_steps = steps / env.curr_episode
else:
    avg_time = (end-start) 
    avg_steps = steps 


if (env.wins + env.losses > 0):
    win_rate = env.wins / (env.wins + env.losses)
else:
    win_rate = 0

print("Avg steps per game: " + str(avg_steps))
print("Total running time: {:.2f} and running time per game: {:.2f}".format(total_time, avg_time) )

## Crear ficheros
file_name = str(env.action_type) + "__" + str(env.reward_mode) + "__" + str(steps) + "__" + str(run)

# METRICS
data_file = open('./metrics/' + file_name + '.txt', 'w')
data_file.write("Games, AvgStepsGame, TotalTime, AvgTimeGame, TotalReward, WinRate \n")
data_file.write("{},{:.2f},{:.2f},{:.2f},{:.2f},{:.2f}".format(env.curr_episode, avg_steps, total_time, avg_time, env.total_reward, win_rate))
data_file.close()

# REWARD DATA
data_file = open('./data/' + file_name + '_reward.csv', 'w')
data_file.write("Games, Reward \n")
for key in env.reward_dict:
    data_file.write("{},{:.2f} \n".format(key, env.reward_dict[key]))
data_file.close()

# Train Games
data_file = open("./data/" + file_name + '_games.csv', 'w')
for l in env.games_outcome:
    data_file.write(str(l) + "\n")
data_file.close()

model.save("./dqn_models/" + file_name + "_dqn")
env.close()
